# Interacting with Articulated Agents

In this tutorial we will show how to interact in Habitat via articulated agents. These are agents composed of different parts which can be articulated. Examples of these agents include different commercial robots (such as Spot, Fetch, Franka) or humanoids.
In this tutorial we will explore how to interact in Habitat with such agents. We will cover the following topics:

- How to initialize an agent
- Moving an agent around the scene
- Dynamic vs Kinematic Simulation
- Interacting with objects
- Interacting with Actions
- Multi-Agent simulation


In [ ]:

import habitat_sim
import magnum as mn
import warnings
from habitat.tasks.rearrange.rearrange_sim import RearrangeSim
warnings.filterwarnings('ignore')
from habitat_sim.utils.settings import make_cfg
from matplotlib import pyplot as plt
from habitat_sim.utils import viz_utils as vut
from omegaconf import DictConfig
import numpy as np
from habitat.articulated_agents.robots import FetchRobot
from habitat.config.default import get_agent_config
from habitat.config.default_structured_configs import ThirdRGBSensorConfig, HeadRGBSensorConfig, HeadPanopticSensorConfig
from habitat.config.default_structured_configs import SimulatorConfig, HabitatSimV0Config, AgentConfig
from habitat.config.default import get_agent_config
import habitat
from habitat_sim.physics import JointMotorSettings, MotionType
from omegaconf import OmegaConf

import git, os
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
data_path = os.path.join(dir_path, "data")
os.chdir(dir_path)

# Initializing a scene with agents
The first thing we want to do is to initialize the simulator to include different agents. 

In the first part of this tutorial we will use `RearrangeSim` as our simulator, which is an abstraction over [HabitatSimulator](https://aihabitat.org/docs/habitat-lab/habitat.core.simulator.Simulator.html) and includes functionalities to update agent cameras and position or interact with objects. In the second part of the tutorial, we will be defining agent actions and will be using a `RearrangeEnvironment`, which contains a reference to the simulator, as well as functions to define and execute agent actions, obtain rewards or termination conditions. The RearrangeEnvironment will also be used to train agents via RL.


## Defining agent configurations
We start by defining a configuration for each agent we want to add. Articulated agents are represented as any other articulated object, and are therefore defined via an URDF file. While this file is enough to represent the agent as an object, it doesn't include a way to easily set its base position, reset its joints, move a specific part or query other attributes.

To simplify this, we provide an abstraction, `ArticulatedAgent`, which will wrap habitat-sim's ManagedArticulatedObject class initialized from the URDF and provide functionalities that are commonly useful for agent control. You can view the different ArticulatedAgents (robots and humanoids) [here](https://github.com/facebookresearch/habitat-lab/tree/main/habitat-lab/habitat/articulated_agents).

In [ ]:
# Define the agent configuration
main_agent_config = AgentConfig()
urdf_path = os.path.join(data_path, "robots/hab_fetch/robots/hab_fetch.urdf")
main_agent_config.articulated_agent_urdf = urdf_path
main_agent_config.articulated_agent_type = "FetchRobot"

# Define sensors that will be attached to this agent, here a third_rgb sensor and a head_rgb.
# We will later talk about why we are giving the sensors these names
main_agent_config.sim_sensors = {
    "third_rgb": ThirdRGBSensorConfig(),
    "head_rgb": HeadRGBSensorConfig(),
}

# We create a dictionary with names of agents and their corresponding agent configuration
agent_dict = {"main_agent": main_agent_config}


In [ ]:
def make_sim_cfg(agent_dict):
    # Start the scene config
    sim_cfg = SimulatorConfig(type="RearrangeSim-v0")
    
    # This is for better graphics
    sim_cfg.habitat_sim_v0.enable_hbao = True
    sim_cfg.habitat_sim_v0.enable_physics = True

    
    # Set up an example scene
    sim_cfg.scene = os.path.join(data_path, "hab3_bench_assets/hab3-hssd/scenes/103997919_171031233.scene_instance.json")
    sim_cfg.scene_dataset = os.path.join(data_path, "hab3_bench_assets/hab3-hssd/hab3-hssd.scene_dataset_config.json")
    sim_cfg.additional_object_paths = [os.path.join(data_path, 'objects/ycb/configs/')]

    
    cfg = OmegaConf.create(sim_cfg)

    # Set the scene agents
    cfg.agents = agent_dict
    cfg.agents_order = list(cfg.agents.keys())
    return cfg


def init_rearrange_sim(agent_dict):
    # Start the scene config
    sim_cfg = make_sim_cfg(agent_dict)    
    cfg = OmegaConf.create(sim_cfg)
    
    # Create the scene
    sim = RearrangeSim(cfg)

    # This is needed to initialize the agents
    sim.agents_mgr.on_new_scene()

    # For this tutorial, we will also add an extra camera that will be used for third person recording.
    camera_sensor_spec = habitat_sim.CameraSensorSpec()
    camera_sensor_spec.sensor_type = habitat_sim.SensorType.COLOR
    camera_sensor_spec.uuid = "scene_camera_rgb"

    # TODO: this is a bit dirty but I think its nice as it shows how to modify a camera sensor...
    sim.add_sensor(camera_sensor_spec, 0)

    return sim


## Initializing the scene
We can now initialize the scene. As mentioned before, we will be using here `RearrangeSim` to easily be able to interact with objects.

We create a scene init function that will take as input a dictionary of agent configurations, as the one we defined before.

In [ ]:
sim = init_rearrange_sim(agent_dict)

We just initialized our scene! We can now query and set our agent position

In [ ]:
init_pos = mn.Vector3(-5.5,0,-1.5)
art_agent = sim.articulated_agent
# We will see later about this
art_agent.sim_obj.motion_type = MotionType.KINEMATIC
print("Current agent position:", art_agent.base_pos)
art_agent.base_pos = init_pos 
print("New agent position:", art_agent.base_pos)
# We take a step to update agent position
_ = sim.step({})

We can also take observations in the environment. Here we get three sensors, two of which we defined in the config and one which we added afterwards.

In [ ]:
observations = sim.get_sensor_observations()
print(observations.keys())

In [ ]:
_, ax = plt.subplots(1,len(observations.keys()))

for ind, name in enumerate(observations.keys()):
    ax[ind].imshow(observations[name])
    ax[ind].set_axis_off()
    ax[ind].set_title(name)

We will now pick the object. In this example, we will directly attach the object to the robot arm, without animating the arm in any way. We also provide a way to train policies so that the arm approaches the object.

# Defining agent actions
So far, we have been controlling agents by directly updating the robot parameters. In  many cases, you may want to abstract interaction into actions that update the robot. These actions can then be called by a planner or a learned policy. In this section we will show how to define and control agents with these actions. The Habitat Quickstart provides more instructions into how to add actions https://aihabitat.org/docs/habitat-lab/quickstart.html.
TODO: point to skills tutorial

To execute actions, we will be using the `Env`, which is an object that contains a simulator instance as well as a set of action definitions and specifiable rewards. We will not be going through the specifiable rewards. 

## Defining an environment
We will start by defining the environment class. A key difference is that now we also define actions that the environment will have

In [ ]:

from habitat.config.default_structured_configs import TaskConfig, EnvironmentConfig, DatasetConfig, HabitatConfig
from habitat.config.default_structured_configs import ArmActionConfig, BaseVelocityActionConfig, OracleNavActionConfig, ActionConfig
from habitat.core.env import Env
def make_sim_cfg(agent_dict):
    # Start the scene config
    sim_cfg = SimulatorConfig(type="RearrangeSim-v0")
    
    # Enable Horizon Based Ambient Occlusion (HBAO) to approximate shadows.
    sim_cfg.habitat_sim_v0.enable_hbao = True
    
    sim_cfg.habitat_sim_v0.enable_physics = True

    
    # Set up an example scene
    sim_cfg.scene = os.path.join(data_path, "hab3_bench_assets/hab3-hssd/scenes/103997919_171031233.scene_instance.json")
    sim_cfg.scene_dataset = os.path.join(data_path, "hab3_bench_assets/hab3-hssd/hab3-hssd.scene_dataset_config.json")
    sim_cfg.additional_object_paths = [os.path.join(data_path, 'objects/ycb/configs/')]

    
    cfg = OmegaConf.create(sim_cfg)

    # Set the scene agents
    cfg.agents = agent_dict
    cfg.agents_order = list(cfg.agents.keys())
    return cfg

def make_hab_cfg(agent_dict, action_dict):
    sim_cfg = make_sim_cfg(agent_dict)
    task_cfg = TaskConfig(type="RearrangeEmptyTask-v0")
    task_cfg.actions = action_dict
    env_cfg = EnvironmentConfig()
    dataset_cfg = DatasetConfig(type="RearrangeDataset-v0", data_path="data/hab3_bench_assets/episode_datasets/small_large.json.gz")
    
    
    hab_cfg = HabitatConfig()
    hab_cfg.environment = env_cfg
    hab_cfg.task = task_cfg
    hab_cfg.dataset = dataset_cfg
    hab_cfg.simulator = sim_cfg
    hab_cfg.simulator.seed = hab_cfg.seed

    return hab_cfg

def init_rearrange_env(agent_dict, action_dict):
    hab_cfg = make_hab_cfg(agent_dict, action_dict)
    res_cfg = OmegaConf.create(hab_cfg)
    return Env(res_cfg)

# Multi-Agent Interaction
So far, we've been executing actions with a single agent. Habitat allows multi-agent execution, we will be looking here at how to do it 

In [ ]:
# We will download spot to show interaction between the spot robot and fetch
# ! python -m habitat_sim.utils.datasets_download --uids hab_spot_arm --no-replace

In [ ]:
# The main difference is in how we define the agent_dict.
# Important: When using more than one agent, we should call them agent_{idx} with idx being between 0 and
# the number of agents. This is required so that we can parse actions
import copy
second_agent_config = copy.deepcopy(main_agent_config)
second_agent_config.articulated_agent_urdf = os.path.join(data_path, "robots/hab_spot_arm/urdf/hab_spot_arm.urdf")
second_agent_config.articulated_agent_type = "SpotRobot"


agent_dict = {"agent_0": main_agent_config, "agent_1": second_agent_config}
multi_agent_action_dict = {
    "agent_0_oracle_magic_grasp_action": ArmActionConfig(
        type="MagicGraspAction"
    ),
    "agent_0_base_velocity_action": BaseVelocityActionConfig(),
    "agent_0_oracle_coord_action": OracleNavActionConfig(
        type="OracleNavCoordinateAction",
        spawn_max_dist_to_obj=2.0,
        motion_control="base_velocity",
    ),
    "agent_1_oracle_magic_grasp_action": ArmActionConfig(
        type="MagicGraspAction"
    ),
    "agent_1_base_velocity_action": BaseVelocityActionConfig(),
    "agent_1_oracle_coord_action": OracleNavActionConfig(
        type="OracleNavCoordinateAction",
        spawn_max_dist_to_obj=2.0,
        motion_control="base_velocity_non_cylinder",
        navmesh_offset=[[0.0, 0.0], [0.25, 0.0], [-0.25, 0.0]],
    ),
}

env = init_rearrange_env(agent_dict, multi_agent_action_dict)

#### The environment takes care about adding prefixes to observations, so that you can query the observation of each of the agents

In [ ]:
observations = env.reset()
_, ax = plt.subplots(1,len(observations.keys()))

for ind, name in enumerate(observations.keys()):
    ax[ind].imshow(observations[name])
    ax[ind].set_axis_off()
    ax[ind].set_title(name)

As before, we can call actions on the agents, prepending the agent_name before the action

In [ ]:
env.reset()
rom = env.sim.get_rigid_object_manager()
# env.sim.articulated_agent.base_pos = init_pos
# As before, we get a navigation point next to an object id

obj_id = env.sim.scene_obj_ids[0]
first_object = rom.get_object_by_id(obj_id)

object_trans = first_object.translation
observations = []

# Walk towards the object

agent_displ = np.inf
agent_rot = np.inf
while agent_displ > 1e-9 or agent_rot > 1e-9:
    prev_rot_0 = env.sim.agents_mgr[0].articulated_agent.base_rot
    prev_pos_0 = env.sim.agents_mgr[0].articulated_agent.base_pos
    prev_rot_1 = env.sim.agents_mgr[1].articulated_agent.base_rot
    prev_pos_1 = env.sim.agents_mgr[1].articulated_agent.base_pos
    action_dict = {
        "action": ("agent_0_oracle_coord_action", "agent_1_oracle_coord_action"), 
        "action_args": {
              "agent_0_oracle_nav_lookat_action": object_trans,
              "agent_0_mode": 1,
              "agent_1_oracle_nav_lookat_action": object_trans,
              "agent_1_mode": 1
          }
    }
    observations.append(env.step(action_dict))
    
    cur_rot_0 = env.sim.agents_mgr[0].articulated_agent.base_rot
    cur_pos_0 = env.sim.agents_mgr[0].articulated_agent.base_pos
    cur_rot_1 = env.sim.agents_mgr[1].articulated_agent.base_rot
    cur_pos_1 = env.sim.agents_mgr[1].articulated_agent.base_pos
    agent_displ_0 = (cur_pos_0 - prev_pos_0).length()
    agent_rot_0 = np.abs(cur_rot_0 - prev_rot_0)
    agent_displ_1 = (cur_pos_1 - prev_pos_1).length()
    agent_rot_1 = np.abs(cur_rot_1 - prev_rot_1)
    agent_displ = max(agent_displ_0, agent_displ_1)
    agent_rot = max(agent_rot_0, agent_rot_1)
    # print(agent_rot, agent_displ)
vut.make_video(
    observations,
    "agent_1_third_rgb",
    "color",
    "robot_tutorial_video",
    open_vid=True,
)